- I highly recommend seting up a Sandbox MongoDB on mlab for minimal hassle. 500MB free should be enough to be going along with - data from completed calculations does not take up much space because all the required data is parsed at the point at which the calculation finishes. For example, around 150 structure optimisation calcs requires ~40MB. 
- To get this example working, you need to install atomate. Dependencies include pymatgen and fireworks so if using `pip`/`conda` this should be taken care of. https://atomate.org/installation.html#install-python-packages

In [1]:
import json
from pymatgen import Structure
from atomate.vasp.workflows.presets.core import wf_structure_optimization
from atomate.vasp.powerups import add_modify_incar, add_additional_fields_to_taskdocs, add_small_gap_multiply, add_priority
from fireworks import LaunchPad

In [4]:
# Import structrures from json
with open('data/example_input_structures.json', 'r') as f:
    structures = json.load(f)
    
# Convert to to pymatgen Structure objects
structures = [{'structure': Structure.from_dict(i['structure']), 'prototype_number': i['prototype_number']} 
                             for i in structures]

print(len(structures))

100


In [5]:
# Connect to launchpad (need to input details for your mongoDB
launchpad = LaunchPad(host="<<HOST>>", port=12345,
                      username="<<USER>>", password="<<PASSWORD>>", 
                      name="<<DB-NAME>>")

In [6]:
# unique indentifiers
for n, i in enumerate(structures):
    i['unique_id'] = 'anion_redox_michael_production_31Jul19_{0}'.format(n)

# Add each structure to the launchpad as a modified workflow 
for i in structures:
    struct = i['structure']
    orig_wf = wf_structure_optimization(struct)
    modified_wf = add_modify_incar(orig_wf, modify_incar_params={'incar_update':
                                                                 { 'NWRITE': 2, 'ISTART': 1, 'INIWAV': 1,
                                                                 'NCORE': 11, 'KPAR': 2,
                                                                 'PREC': 'Accurate', 'ALGO': 'All',
                                                                  'ENCUT': 680, 'NELM': 200,
                                                                  'EDIFF': 1E-6, 'GGA': 'PS',
                                                                  'LASPH': '.TRUE.', 'LREAL': 'Auto',
                                                                  'ADDGRID': '.TRUE.', 'EDIFFG': -0.01,
                                                                  'NSW': 400, 'IBRION': 2,
                                                                  'ISIF': 3, 'ISYM': 2,
                                                                  'ISMEAR': 0, 'SIGMA': 0.05,
                                                                  'LORBIT': 11, 'NEDOS': 2000 }})
                                                                 
    modified_wf = add_additional_fields_to_taskdocs(modified_wf, 
                                                update_dict={'unique_id': i['unique_id'], 
                                                             'prototype_number': i['prototype_number']})
    
    modified_wf = add_priority(modified_wf, 100)

    launchpad.add_wf(modified_wf)

2019-07-31 14:17:50,675 INFO Added a workflow. id_map: {-1: 12}
2019-07-31 14:17:50,750 INFO Added a workflow. id_map: {-2: 13}
2019-07-31 14:17:50,822 INFO Added a workflow. id_map: {-3: 14}
2019-07-31 14:17:50,892 INFO Added a workflow. id_map: {-4: 15}
2019-07-31 14:17:50,964 INFO Added a workflow. id_map: {-5: 16}
2019-07-31 14:17:51,035 INFO Added a workflow. id_map: {-6: 17}
2019-07-31 14:17:51,108 INFO Added a workflow. id_map: {-7: 18}
2019-07-31 14:17:51,189 INFO Added a workflow. id_map: {-8: 19}
2019-07-31 14:17:51,260 INFO Added a workflow. id_map: {-9: 20}
2019-07-31 14:17:51,331 INFO Added a workflow. id_map: {-10: 21}
2019-07-31 14:17:51,403 INFO Added a workflow. id_map: {-11: 22}
2019-07-31 14:17:51,474 INFO Added a workflow. id_map: {-12: 23}
2019-07-31 14:17:51,545 INFO Added a workflow. id_map: {-13: 24}
2019-07-31 14:17:51,617 INFO Added a workflow. id_map: {-14: 25}
2019-07-31 14:17:51,687 INFO Added a workflow. id_map: {-15: 26}
2019-07-31 14:17:51,759 INFO Added